<a href="https://colab.research.google.com/github/profcomff/chatbot-mark-api/blob/main/notebooks/Hitrate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/profcomff/chatbot-mark-api.git

Cloning into 'chatbot-mark-api'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 224 (delta 16), reused 36 (delta 11), pack-reused 178 (from 1)
Receiving objects: 100% (224/224), 2.38 MiB | 5.89 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [2]:
CHROMA_DIR = "/content/chatbot-mark-api/chroma_db"

topk = 5
retrivier_k = 4
ensemble_k=8

# Библиотеки

In [3]:
!pip install langchain transformers sentence-transformers -q
!pip install -U langchain-community -q
!pip install -qU "langchain-chroma>=0.1.2" -q
!pip install langchain_huggingface -q
!pip install rank_bm25 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/

In [4]:
# Core Python libraries
import re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import pickle

from langchain_core.embeddings import Embeddings
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import torch

# Torch and Transformers
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import (
    AutoModelForSequenceClassification,
    AutoModel,
    AutoTokenizer,
    XLMRobertaTokenizer,
    XLMRobertaModel,
)

# NLTK for text preprocessing
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords') #добалено

# Sentence Transformers
from sentence_transformers import CrossEncoder

# LangChain - documents and embeddings
from langchain_core.documents import Document
from langchain_core.embeddings import Embeddings
from langchain_core.vectorstores import InMemoryVectorStore


# LangChain - loading, splitting, and vector stores
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma

from langchain.embeddings import HuggingFaceEmbeddings #this?
from langchain_huggingface import HuggingFaceEmbeddings #or this?

# LangChain - retrieval and reranking
from langchain.retrievers import (
    ContextualCompressionRetriever,
    EnsembleRetriever
)
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.retrievers import BM25Retriever

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
import sys
sys.path.append("/content/chatbot-mark-api")

from nn.search import get_context, E5LangChainEmbedder, preprocess

# Функции для расчета метрик

In [6]:
def calculate_hitrate_langchain(retriever, questions_df, topk=5, topic_column='topic_name', print_results=True):
    """
    Вычисляет hitrate — долю вопросов, для которых правильный топик есть среди top-k результатов.
    Также выводит подробную информацию по количеству вопросов и попаданий.

    Args:
        retriever: LangChain Retriever с методом get_relevant_documents(query)
        questions_df: pd.DataFrame с колонками 'question' и topic_column (например, 'topik')
        topk: сколько документов брать для подсчёта hitrate
        topic_column: имя колонки с правильным названием топика

    Returns:
        float: hitrate (0..1)
    """
    hits = 0
    total = len(questions_df)

    for _, row in questions_df.iterrows():
        query = row['question']
        correct_topic = row[topic_column]

        docs = retriever.get_relevant_documents(query)[:topk]
        found_topics = [doc.metadata.get('source', '') for doc in docs]

        if correct_topic in found_topics:
            hits += 1

    hitrate = hits / total if total > 0 else 0

    if print_results:
        print(f"Total questions checked: {total}")
        print(f"Correct hits in top-{topk}: {hits}")
        print(f"Hitrate: {hitrate:.3f} ({hitrate*100:.1f}%)")

    return hitrate

In [19]:
def find_best_ensemble_weights(
    bm25_retriever,
    vector_store,
    questions_df,
    topk=5,
    retrivier_k=4,
    ensemble_k=8,
    weights_grid=None,
    topic_column='topic_name',
    verbose=True
):
    """
    Перебирает веса для ансамбля BM25 + vector retriever,
    вычисляет hitrate и возвращает лучшие веса и hitrate.

    Args:
        bm25_retriever: BM25 retriever
        vector_store: vector store (с методом as_retriever)
        questions_df: DataFrame с вопросами и правильными топиками
        topk: сколько документов брать для подсчёта hitrate
        retrivier_k: сколько документов искать в каждом retriever
        ensemble_k: сколько документов возвращает ансамблевый retriever
        weights_grid: список float для перебора веса bm25 (vector вес = 1 - bm25)
        topic_column: имя колонки с правильным топиком
        verbose: выводить ли прогресс

    Returns:
        tuple: (best_weights, best_hitrate)
    """
    if weights_grid is None:
        weights_grid = [0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]

    vector_retriever = vector_store.as_retriever(search_kwargs={"k": retrivier_k})

    best_hitrate = 0
    best_weights = None

    for bm25_weight in tqdm(weights_grid): #tqdm
        vector_weight = 1.0 - bm25_weight
        weights = [bm25_weight, vector_weight]

        ensemble_retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, vector_retriever],
            weights=weights,
            k=ensemble_k
        )

        hitrate = calculate_hitrate_langchain(
            retriever=ensemble_retriever,
            questions_df=questions_df,
            topk=topk,
            topic_column=topic_column
        )

        if verbose:
            print(f"Weights: {weights}, Hitrate: {hitrate:.2%}\n")

        if hitrate > best_hitrate:
            best_hitrate = hitrate
            best_weights = weights

    if verbose:
        print(f"\nBest Weights: {best_weights}, Best Hitrate: {best_hitrate:.2%}")

    return best_weights, best_hitrate


# Загрузка модели и данных


In [8]:
tokenizer = XLMRobertaTokenizer.from_pretrained("d0rj/e5-base-en-ru", use_cache=False)
model = XLMRobertaModel.from_pretrained("d0rj/e5-base-en-ru", use_cache=False)

embedder = E5LangChainEmbedder(
    tokenizer=tokenizer,
    model=model,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/471 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/529M [00:00<?, ?B/s]

In [9]:
vector_store = Chroma(
        collection_name="docs",
        embedding_function=embedder,
        persist_directory=CHROMA_DIR # db_path = "/content/chatbot-mark-api/chroma_db"
    )

In [10]:
all_docs = vector_store.get(include=["documents", "metadatas"])

documents = [
    Document(page_content=doc_text, metadata=metadata)
    for doc_text, metadata in zip(all_docs["documents"], all_docs["metadatas"])
]

documents[0:2]

[Document(metadata={'source': 'Карта зачет'}, page_content='Карта зачет. https://vk.com/wall-24234717_22977\nЭто ваш профсоюзный билет. С помощью этой карты вы можете получать скидки у полезных для студентов популярных брендов, участвовать в конкурсах и розыгрышах, а также посещать концерты и мероприятия. \nПолный перечень скидок есть в статье: vk.cc/bYSCNw.'),
 Document(metadata={'source': 'Как вступить в профсоюз'}, page_content='Как вступить в профсоюз? Чтобы вступить в Профсоюз, можно прийти в кабинет Профкома (2-39) в рабочие часы (11:00-16:00) или воспользоваться сервисом в приложении «Твой ФФ» и подать заявление на сайте. Подписанное заявление также необходимо отнести в кабинет Профкома.')]

In [11]:
bm25_retriever = BM25Retriever.from_documents(
    documents,
    preprocess_func=preprocess #
)

In [12]:
#debug
results, combined_text = get_context(
    query="как восстановиться после отчисления",
    tokenizer=tokenizer,
    model=model,
    bm_25=bm25_retriever,
    vector_store=vector_store,
    ensemble_k=ensemble_k, #?
    retrivier_k=retrivier_k #?
)

display(results)
# display(combined_text)

[{'topic': 'Вызов на КСД',
  'full_text': 'Вызов на КСД. Что делать, если вызвали на КСД? Для начала вам следует понять, что никто на комиссии не хочет вашего отчисления, напротив, вам всегда хотят помочь. Поэтому ваша главная задача – не доводить до ситуации, когда вам уже нельзя помочь. Для этого есть всего три простых совета:\n\n— Если вы заболели и пропустили занятия, зачёт или экзамен, получите в поликлинике подтверждающую справку и принесите её инспектору курса. Лежащая в личном деле справка поможет вам сохранить попытку сдачи, так как причина неявки уважительная. Обратите внимание, что справку необходимо предоставлять в учебную часть своевременно, не позднее трёх дней со дня получения!\n\n— Всегда сообщайте в учебную часть, если у вас появились обстоятельства, препятствующие освоению учебной программы. Ваш начальник курса может помочь советом, а если ситуация зайдёт далеко, вам будет проще, например, взять академический отпуск.\n\n— Если вы не хотите идти в учебную часть со свое

In [14]:
# questions_deep = pd.read_excel("/content/chatbot-mark-api/file/questions_v2.xlsx")
questions_deep = pd.read_excel("/content/chatbot-mark-api/file/questions_v2_deep.xlsx")

questions_short = pd.read_excel("/content/chatbot-mark-api/file/questions_v2_short.xlsx")

display(questions_deep.head(2))

,topic_name,question,id
0,Карта зачет,Какие преимущества предоставляет профсоюзный б...,0
1,Карта зачет,"Какую информацию можно найти по ссылке, указан...",0


# Расчет метрик на длинных вопросах

In [15]:
retriever = bm25_retriever
questions_df = questions_deep

display(retriever)

hitrate = calculate_hitrate_langchain(retriever, questions_df, topk=3)

BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7a10521f7f10>, preprocess_func=<function preprocess at 0x7a105217d260>)

/tmp/ipython-input-6-805641044.py:22: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)[:topk]


Total questions checked: 525
Correct hits in top-3: 455
Hitrate: 0.867 (86.7%)


In [16]:
best_weights, best_hitrate = find_best_ensemble_weights(
    bm25_retriever=bm25_retriever,
    vector_store=vector_store,
    questions_df=questions_df,
    topk=topk,
    retrivier_k=retrivier_k,
    ensemble_k=ensemble_k,
    topic_column='topic_name',
    verbose=True
)

Total questions checked: 525
Correct hits in top-5: 508
Hitrate: 0.968 (96.8%)
Weights: [0.0, 1.0], Hitrate: 96.76%
Total questions checked: 525
Correct hits in top-5: 508
Hitrate: 0.968 (96.8%)
Weights: [0.1, 0.9], Hitrate: 96.76%
Total questions checked: 525
Correct hits in top-5: 508
Hitrate: 0.968 (96.8%)
Weights: [0.25, 0.75], Hitrate: 96.76%
Total questions checked: 525
Correct hits in top-5: 514
Hitrate: 0.979 (97.9%)
Weights: [0.5, 0.5], Hitrate: 97.90%
Total questions checked: 525
Correct hits in top-5: 509
Hitrate: 0.970 (97.0%)
Weights: [0.75, 0.25], Hitrate: 96.95%
Total questions checked: 525
Correct hits in top-5: 509
Hitrate: 0.970 (97.0%)
Weights: [0.9, 0.09999999999999998], Hitrate: 96.95%
Total questions checked: 525
Correct hits in top-5: 509
Hitrate: 0.970 (97.0%)
Weights: [1.0, 0.0], Hitrate: 96.95%

Best Weights: [0.5, 0.5], Best Hitrate: 97.90%


# Расчет метрик на коротких вопросах

In [17]:
retriever = bm25_retriever
questions_df = questions_short

display(retriever)

hitrate = calculate_hitrate_langchain(retriever, questions_df, topk=3)

BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7a10521f7f10>, preprocess_func=<function preprocess at 0x7a105217d260>)

Total questions checked: 525
Correct hits in top-3: 405
Hitrate: 0.771 (77.1%)


In [20]:
best_weights, best_hitrate = find_best_ensemble_weights(
    bm25_retriever=bm25_retriever,
    vector_store=vector_store,
    questions_df=questions_df,
    topk=topk,
    retrivier_k=retrivier_k,
    ensemble_k=ensemble_k,
    topic_column='topic_name',
    verbose=True
)

  0%|          | 0/7 [00:00<?, ?it/s]

Total questions checked: 525
Correct hits in top-5: 470
Hitrate: 0.895 (89.5%)
Weights: [0.0, 1.0], Hitrate: 89.52%

Total questions checked: 525
Correct hits in top-5: 470
Hitrate: 0.895 (89.5%)
Weights: [0.1, 0.9], Hitrate: 89.52%

Total questions checked: 525
Correct hits in top-5: 470
Hitrate: 0.895 (89.5%)
Weights: [0.25, 0.75], Hitrate: 89.52%

Total questions checked: 525
Correct hits in top-5: 474
Hitrate: 0.903 (90.3%)
Weights: [0.5, 0.5], Hitrate: 90.29%

Total questions checked: 525
Correct hits in top-5: 465
Hitrate: 0.886 (88.6%)
Weights: [0.75, 0.25], Hitrate: 88.57%

Total questions checked: 525
Correct hits in top-5: 465
Hitrate: 0.886 (88.6%)
Weights: [0.9, 0.09999999999999998], Hitrate: 88.57%

Total questions checked: 525
Correct hits in top-5: 465
Hitrate: 0.886 (88.6%)
Weights: [1.0, 0.0], Hitrate: 88.57%


Best Weights: [0.5, 0.5], Best Hitrate: 90.29%
